In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os
from sklearn import neighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [3]:
imagefiles = ['farm1.jpg', 'farm2.jpg', 'farm3.jpg', 'farm4.jpg',
 'farm5.jpg', 'farm6.jpg', 'farm7.jpg', 'farm8.jpg',
 'city1.jpg', 'city2.jpg', 'city3.jpg', 'city4.jpg',
 'city5.jpg', 'city6.jpg', 'city7.jpg', 'city8.jpg',
 'desert1.jpg', 'desert2.jpg', 'desert3.jpg', 'desert4.jpg',
 'desert5.jpg', 'desert6.jpg', 'desert7.jpg', 'desert8.jpg']

testfiles = ["test1.jpg","test2.jpg","test3.jpg"]

training_target = ['farm', 'farm', 'farm', 'farm',
 'farm', 'farm', 'farm', 'farm',
 'city', 'city', 'city', 'city',
 'city', 'city', 'city', 'city',
 'desert', 'desert', 'desert', 'desert',
 'desert', 'desert', 'desert', 'desert']

test_Actual = ['city','desert','farm']

training_data = np.zeros((24,2))

testing_data = np.zeros((3,2))

r = 0

def imageload(filename):
    file = cv2.imread(filename)
    file = cv2.cvtColor(file, cv2.COLOR_BGR2RGB)
    return file

def datageneration(image,split,r):
    pic = imageload(image)
    rehshaped_pic = pic.reshape(pic.shape[0]*pic.shape[1], 3)
    classifier = KMeans(n_clusters = 1)
    labels = classifier.fit_predict(rehshaped_pic)
    labelcounts = Counter(labels)
    center_colors = classifier.cluster_centers_
    for key in labelcounts.keys():
        rgb_colors = [center_colors[key]]
        rgb_sum = rgb_colors[0][0]+rgb_colors[0][1]+rgb_colors[0][2]
        green_perc = rgb_colors[0][1]/rgb_sum
        blue_perc = rgb_colors[0][2]/rgb_sum
        if split=="train":
            training_data[r,0] = green_perc
            training_data[r,1] = blue_perc
        elif split=="test":
            testing_data[r,0] = green_perc
            testing_data[r,1] = blue_perc
       
    if split=="train":
        return training_data
    if split=="test":
        return testing_data
    
r = 0
        
#training data
for image in imagefiles:
    train_data = datageneration(image,"train",r)
    r = r+1

k=0
#test data
for file in testfiles:
    test_data = datageneration(file,"test",k)
    k = k+1
    
#Decision tree classifier    
clf = DecisionTreeClassifier()

clf = clf.fit(train_data,training_target)

test_predicted = clf.predict(test_data)

print("Predicted test data is "+str(test_predicted))

print("Actual test data is "+str(test_Actual))

print("Accuracy:",metrics.accuracy_score(test_Actual, test_predicted))



    
    


Predicted test data is ['city' 'desert' 'desert']
Actual test data is ['city', 'desert', 'farm']
Accuracy: 0.6666666666666666


In [4]:
#Knn Claffier
knn_classifier = neighbors.KNeighborsClassifier(1,weights='distance')

knn_classifier.fit(training_data,training_target)

test_pred1 = knn_classifier.predict(test_data)

print("Predicted test data is "+str(test_pred1))

print("Actual test data is "+str(test_Actual))

Predicted test data is ['city' 'desert' 'desert']
Actual test data is ['city', 'desert', 'farm']
